# What happens when we assign a value to a variable?

In [1]:
a = 10

- Now the variable `a` points to an object in memory
    - The object in memory happens to be an integer with value 10
        - We say that **the variable name is bound to the object**

- By using the variable name in our code, we can access the object it was bound to
    - However, **we can't just reference `a` ANYWHERE in our code!**
        - Maybe that label only exists in a certain scope
        - Maybe that label points elsewhere in another scope

- The part of the code where the variable points to the object is called the **lexical scope** (aka scope)
    - The bindings between the variables and the memory addresses are stored in **namespaces**
        - We can think of this as a table with:
            1. Column 1: Names
            2. Column 2: Addresses

- Every scope has a corresponding namespace
    - They go hand-in-hand

# What is the global scope?

- This is essentially the module scope
    - It spans a single file
        - In Python, there's no such thing as a truly global scope
            - The only exception is the built-in scope

- Built-in and global variables can be used **anywhere** inside our module

- Global scopes are nested inside the built-in scope

![](images/nested_scopes.PNG)

- Let's say we reference some variable in Module 1
    - If the variable doesn't exist inside the Module 1 name space, the code will next look in the built-in scope's namespace

### Examples

**Example 1**

- Let's say we have a file saved called module1.py
    - There's only one line in it: `print(True)`
    
- Let's actually create the file

In [3]:
with open('module1.py', 'w') as f: 
    f.write('print(True)') 

- Neither `print` nor `True` are defined in the module
    - Therefore, they won't be found in the namespace when the code is run

In [4]:
%run module1

True


- As we can see, it didn't fail when we ran it
    - When we ran the code, it looked for `print` and `True` in the built-in scope

**Example 2**

- Now, let's say module2.py contains `print(a)`
    - Unlike the previous example, this will cause an error

In [5]:
with open('module2.py', 'w') as f: 
    f.write('print(a)') 

In [6]:
%run module2

NameError: name 'a' is not defined

- As expected, we get an error
    - This is because `a` isn't defined in the module2 or the built-in namespace

**Example 3**

- In module3.py, we have:

```python
print = lambda x: f'hello {x}'
s = print('world')
```

- We overwrote the built-in `print` function with a custom one
    - Instead of printing it, it creates a string

In [8]:
code = """
print = lambda x: f'hello {x}'
s = print('world')
"""

with open('module3.py', 'w') as f: 
    f.write(code) 

In [9]:
%run module3

- As we can see, nothing was printed
    - This is what we expected
        - Within the module3 namespace, `s` was be defined as `'hello world'`

- **Note**: overwriting built-in functions with custom ones is called **masking**
    - Not recommended

# What is the local scope?

- When we define functions, we can create variables inside the function

```python
def f():
    a = 10
    print(a)
```

- Inside the function, we now have a variable `a`
    - The actual variable **isn't created until the function is called**

- Every time the function is called, a new scope is created!
    - That's why recursion works
        - It's not the same object being fed into itself

In [73]:
def my_func(a, b):
    c = a * b
    return c

- When the Python code is **compiled**:
    - `my_func` is added to the name space and points to the function object
    - `a`, `b`, and `c` are determined to be local variables
        - Therefore, they aren't added to the namespace
    - A namespace for `my_func` hasn't been created yet
        - That happens when we run the function

- Now, let's say we call the function:

In [74]:
my_func('z',2)

'zz'

- At **runtime**, a namespace for `my_func` is created
    - `a` and `b` are added to the name space, and they point to 'z' and 2
        - Furthermore, `c` is created and points to 'zz'

- *What happens once the function is finished?*
    - The scope is deleted

- Now, let's say we call the function again:

In [75]:
my_func(10, 5)

50

- Now, a **new** scope is created, and `a`, `b` and `c` are defined within it
    - Scope is deleted when the function is finished

# What are nested scopes?

- Let's consider a russian nesting doll of scopes:

![](images/nested_scopes_2.PNG)

- When we request an object bound to a variable name:
    - E.g. `print(a)`
        - The order of looking for `a` is:
            1. local scope
            2. module scope
            3. built-in scope

- When a function is finished, the scope is deleted
    - Therefore, for each memory adress that is referenced in the function, the reference count goes down by one when the scope is deleted
        - When this happens, we say that the variable **goes out of scope**

# How do we modify a global variable from inside a function?

**Example**

In [84]:
a = 0

In [85]:
def my_func():
    a = 100
    print(a)

- When `my_func` is compiled, the local scope isn't created
    - Therefore, as far as our code is concerned, **there's only one `a` and it's equal to 0**

- Inside our function, the local variable `a` **masks** the global version

In [86]:
my_func(), a

100


(None, 0)

- As we can see, the local version of `a` is equal to 100, and the global version is equal to 0

- Let's say we want to change the global version of `a` from inside the function
    - We can used the keyword `global`

In [87]:
a = 0

def my_func():
    global a
    a = 100
    print(a)

In [88]:
my_func(), a

100


(None, 100)

- As we can see, the `global` keyword updated the global version of `a`